In [20]:
import os 
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import torch 
from torchvision.utils import save_image
from src.data.utils import hsl2rgb




In [22]:
# get rectangle images
samples = []
labels = [1, 33, 60, 150, 220, 254]
X_rectangles = np.load(file="../data/sim_cont/coxph/X_train.npy")
df_train = pd.read_csv("../data/sim_cont/coxph/df_train.csv")
for i in range(X_rectangles.shape[0]):
    if df_train.iloc[df_train['labels'], i] in labels:
        while len(samples) < 6:
            x = X_rectangles[i]
            x = np.transpose(x, axes=(1, 2, 0))
            x = hsl2rgb(x)
            x = np.transpose(x, axes=(2, 0, 1))
            samples.append(x)
    else:
        continue

    # x = np.transpose(x, axes=(1, 2, 0))
    # plt.imshow(x)

samples = np.stack(samples)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [18]:
samples = torch.Tensor(samples)
save_image(samples, "rectangles.png", nrow=1)

In [6]:
X_mult = np.load(file="../data/sim_cont_mult/coxph/X_train.npy")